# TASK 3: Evaluation von Input-Techniken

In dieser Studie werden verschiedene Eingabemethoden miteinander verglichen. Dazu zählt neben Maus und Touchpad auch eine gestenbasierte Eingabe. Außerdem wurde die Auswirkung von Latenz in Betracht gezogen. Dafür wurde reziproke Antippaufgabe verwendet. 

## Methodik

### Apperatus

Die Teilnehmer führten die Studie auf einem Macbook Air M1 2020 durch. Für die Eingabe wurden folgende Methoden verwendet:
- Maus (Logitech MX Anywhere 3)
- Maus (Logitech MX Anywhere 3) mit 150ms Latenz
- Touchpad (vom Macbook)
- gestenbasierte Eingabe durch eine Hand

Die gestenbasierte Eingabe erfolgte über die Handerkennung der Google MediaPipe. Dabei konnte der Mauszeiger über die Mitte der Hand (erstes Gelenk des Mittelfingers) gesteuert werden, wobei der Linksklick über die Berührung von Daumen und Zeigefinger erfolgte.

Bei der reziproken Antippaufgabe wurden 9 Kreise in einem UserInterface dargestellt. Dabei wurden für die Breite und den Abstand der Kreise alle Kombinationen aus 3 verschiedenen Werten verwendet. Diese wurden in einem Durchlauf automatisch durch das Programm geändert. Für jede dieser 9 Kombinationen mussten die Teilnehmer pro Gerät je 3 Wiederholungen durchlaufen. Bei jeder Möglichkeit wurden 9 Klicks auf Kreise erfasst. Der erste wurde dabei ignoriert. Damit wurden pro Teilnehmer und Gerät 243 Datenpunkte erfasst.

### Prozedur

Die Teilnehmer wurden begrüßt und gebeten, die Einverständniserklärung sorgfälltig durchzulesen und diese zu unterschreiben. 

Daraufhin sollte ein demographischer Fragebogen ausgefüllt werden, der neben Alter und Geschlecht auch die berufliche Situation und die Computernutzung erfasste.

Danach wurde ihnen das Vorgehen geschildert und die verschiedensten Eingabemethoden genau beschrieben. Ihnen wurde gesagt, dass sie mit verschiedenen Eingabegeräten aus vielen Kreisen jeweils den bunten so schnell wie möglich aber präzise anklicken sollen. Daraufhin wurde die Anwendung von der Versuchsleitung geöffnet und die richtigen Parameter eingestellt. Dem Teilnehmer wurde gesagt, welches Eingabegerät er verwenden soll und nochmals kurz die Nutzung erklärt. Die Erfassung startete dann mit dem ersten Klick. Sobald keine Kreise mehr angezeigt wurden, war die Aufgabe für das Eingabegerät beendet. Dergleiche Vorgang wurde auch für die anderen Eingabemethoden wiederholt. Wir haben die Reihenfolge der Eingabemethoden für jeden Teilnehmer variiert, um mögliche Lerneffekte auszugleichen. 

### Teilnehmer

An unserer Studie nahmen 3 Teilnehmer im Alter von 22 bis 56 Jahren teil (SD = ..., M = ...). Zwei davon identifizierten sich als weiblich, eine Personen als männlich. Alle Teilnehmer gaben an, die Maus als häufigste Eingabemethode zu verwenden. Außerdem hatte bisher keiner der  Teilnehmer Erfahrungen mit gestenbasierter Eingabe.

## Ergebnisse

### Daten einlesen und zusammenführen

In [2]:
import pandas as pd
import os

In [6]:
folder_path = 'study-data'
columns = ['id', 'participant_id', 'input_method', 'latency', 'duration', 'timestamp', 'distance', 'width', 'index_of_difficulty']

In [7]:
data_frames = []

# Iteriere durch die Dateien im Ordner
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        # Vollständiger Pfad zur CSV-Datei
        file_path = os.path.join(folder_path, file_name)
        
        # Lese die CSV-Datei ein
        df = pd.read_csv(file_path, header=None)
        
        # Füge die Spaltenüberschriften hinzu
        df.columns = columns
        
        # Füge den DataFrame zur Liste hinzu
        data_frames.append(df)

# Kombiniere alle DataFrames
combined_df = pd.concat(data_frames, ignore_index=True)

combined_df

,id,participant_id,input_method,latency,duration,timestamp,distance,width,index_of_difficulty
0,2,participant3,mouse,0.00,0.551746,1.719589e+09,275.0,50.0,3.459432
1,3,participant3,mouse,0.00,0.664206,1.719589e+09,275.0,50.0,3.459432
2,4,participant3,mouse,0.00,1.074879,1.719589e+09,275.0,50.0,3.459432
3,5,participant3,mouse,0.00,1.064473,1.719589e+09,275.0,50.0,3.459432
4,6,participant3,mouse,0.00,0.495360,1.719589e+09,275.0,50.0,3.459432
...,...,...,...,...,...,...,...,...,...
2911,266,participant3,mouse_latency,0.15,0.894194,1.719589e+09,200.0,35.0,3.514573
2912,267,participant3,mouse_latency,0.15,0.856843,1.719589e+09,200.0,35.0,3.514573
2913,268,participant3,mouse_latency,0.15,0.797605,1.719589e+09,200.0,35.0,3.514573
2914,269,participant3,mouse_latency,0.15,1.129769,1.719589e+09,200.0,35.0,3.514573


## Probleme

Es stellte sich für die Teilnehmer als schwierig dar, Objekte am Rand per Gesten zu selektieren, da dort die Hand nicht mehr zuverlässig erkannt werden konnte. Für die Zukunft empfielt es sich, das erfasste Kamerabild nicht in derselben Größe auf Bildschirm zu projezieren.

Ein Problem der Studie war es, die verschiedenen Distanzen und Breite der Ziele zu definieren, da wir kein erkennbares muster implementieren wollten. Dafür haben wir die Werte mit einem vorher gesetzten Seed zufällig generiert.

Ein weiteres Problem war das erzeugen einer realistischen Latenz. Da wir für die Versuchsdurchführung Pyglet verwendeten, konnte die Latenz nicht mit einfachen event-handlern simuliert werden, was ein flackern des Zeigers zur Folge hatte.